In [ ]:
%pip install tensorflow-gpu

In [ ]:
import os
import tensorflow as tf
import numpy as np

In [ ]:
path_to_file=tf.keras.utils.get_file('master_chat_bot_data.txt','https://raw.githubusercontent.com/projjal1/datasets/master/master_chat_bot_data.txt')

In [ ]:
text=open(path_to_file,'rb').read().decode(encoding='utf-8')

In [ ]:
print("The length of characters in the text of database",len(text))

The length of characters in the text of database 172545


In [ ]:
#Creating the vocabulary of words
vocab=sorted(set(text))

In [ ]:
print(vocab[:40])

['\n', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']


#Vectorize the text
Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [ ]:
char2idx={u:i for i,u in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '<': 27, '=': 28, '>': 29, '?': 30, 'A': 31, 'B': 32, 'C': 33, 'D': 34, 'E': 35, 'F': 36, 'G': 37, 'H': 38, 'I': 39, 'J': 40, 'K': 41, 'L': 42, 'M': 43, 'N': 44, 'O': 45, 'P': 46, 'Q': 47, 'R': 48, 'S': 49, 'T': 50, 'U': 51, 'V': 52, 'W': 53, 'X': 54, 'Y': 55, 'Z': 56, '[': 57, ']': 58, '^': 59, '_': 60, 'a': 61, 'b': 62, 'c': 63, 'd': 64, 'e': 65, 'f': 66, 'g': 67, 'h': 68, 'i': 69, 'j': 70, 'k': 71, 'l': 72, 'm': 73, 'n': 74, 'o': 75, 'p': 76, 'q': 77, 'r': 78, 's': 79, 't': 80, 'u': 81, 'v': 82, 'w': 83, 'x': 84, 'y': 85, 'z': 86, '⁇': 87, '？': 88, '🙈': 89}


In [ ]:
idx2char=np.array(vocab)
print(idx2char)

['\n' ' ' '!' '"' '#' '%' "'" '(' ')' '*' '+' ',' '-' '.' '/' '0' '1' '2'
 '3' '4' '5' '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' '[' ']' '^' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k'
 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '⁇' '？' '🙈']


In [ ]:
#Convert the text to int
text_as_int=np.array([char2idx[c] for c in text])
print(text_as_int)

[38 81 73 ... 74 13  0]


In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Human 1: Hi!\n' ---- characters mapped to int ---- > [38 81 73 61 74  1 16 25  1 38 69  2  0]


In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Create test and training batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Model building using RNN

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 90) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           23040     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 90)            92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

Add an optimizer and loss function

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 90)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.499277


In [ ]:
model.compile(optimizer='adam',loss=loss,metrics=['accuracy'])

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history=model.fit(dataset, epochs=50,callbacks=[checkpoint_callback])

Epoch 1/50
26/26 [==============================] - 2s 71ms/step - loss: 0.1431 - accuracy: 0.9677
Epoch 2/50
26/26 [==============================] - 2s 70ms/step - loss: 0.1370 - accuracy: 0.9692
Epoch 3/50
26/26 [==============================] - 2s 70ms/step - loss: 0.1316 - accuracy: 0.9709
Epoch 4/50
26/26 [==============================] - 2s 70ms/step - loss: 0.1273 - accuracy: 0.9713
Epoch 5/50
26/26 [==============================] - 2s 69ms/step - loss: 0.1298 - accuracy: 0.9711
Epoch 6/50
26/26 [==============================] - 2s 70ms/step - loss: 0.1307 - accuracy: 0.9709
Epoch 7/50
26/26 [==============================] - 2s 70ms/step - loss: 0.1292 - accuracy: 0.9711
Epoch 8/50
26/26 [==============================] - 2s 69ms/step - loss: 0.1301 - accuracy: 0.9711
Epoch 9/50
26/26 [==============================] - 2s 70ms/step - loss: 0.1303 - accuracy: 0.9708
Epoch 10/50
26/26 [==============================] - 2s 70ms/step - loss: 0.1294 - accuracy: 0.9712
Epoch 11/

Now we will rebuild the model from the saved weigths

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            23040     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 90)             92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


#The fun part
Let's generate text using the model and predictions on the given dataset nature.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  output_string=''.join(text_generated)
  print(output_string)

In [ ]:
s=input("Enter the text: ")
s='Human 1: '+s
generate_text(model, start_string=s)

Enter the text: Where do you live?

Human 2: My location in 2: Wow, thank you. You say tell me what's going on?
Human 2: I'm just really good at procrastinating! :D I've been putting special?
Human 2: That's just they are jumping about in meadows. You love lamb. What do you like to do whendeman 1: About 3 really speak 3olle of the world.
Human 2: That's fantastic! :)
Human 1: How about you?
Human 2: I'm doing pretty good, thanks for asked!
Human 2: are they americans?
Human 1: How many friends do you have?
Human 2: I have thousands of human and robot friends from all over the world. Stay The Beakend to vote which party expraiting to be had.
Human 1: Hi!
Human 2: Hi it.
Human 1: let's speak Check then watch something
Human 2: Whatcha gonna eat?
Human 1: no idea yet, 1: Which is a more useful subject, philosophy or psychology?
Human 2: Psychology, definitely.
Human 1: Who are you?
Human 2: I am Mitsuku. I want to be your friend.
Human 1: Why do you ask?
Human 1: how are you today?
Human 